In [14]:
from truss_casadi import *

joints = [
    Joint(JointType.PIN),  # 0
    Joint(JointType.GUSSET), # 1
    Joint(JointType.ROLLER), # 2 
    Joint(JointType.GUSSET), # 3
    Joint(JointType.GUSSET)] # 4

links = [Link(0, 1), Link(1,2), Link(0,3), Link(1,3), Link(1,4), Link(2,4), Link(3,4)]

FORCE_PER_M = 2.5
train_force = Force(fx=0, fy=FORCE_PER_M, joints=[0, 1, 2])
forces = [train_force]

truss = Truss(joints, links, forces)

cost = Function('cost', [truss.xs], [truss.cost])
jac = Function('jacobian', [truss.xs], [truss.jacobian])
hess = Function('hessian', [truss.xs], [truss.hessian])

xs = [0, 7, 14, 3.5, 10.5]
ys = [0, 0, 0, -20, -20]

print(cost(xs + ys))
print(jac(xs + ys))
print(hess(xs + ys))

2167.82
[[-55.2599, -8.73257e-11, 55.2599, -14.5016, 14.5016, 35.6914, 16.5629, 35.6914, -43.9728, -43.9728]]

[[4741.2, -4457.49, -3.18323e-12, -1.43364, -282.28, -0.626324, 878.858, -830.336, -1659.17, 1611.27], 
 [-4457.49, 8386.55, -4457.49, 264.209, 264.209, -781.814, 1.89806e-09, 781.814, 3075.76, -3075.76], 
 [-3.18323e-12, -4457.49, 4741.2, -282.28, -1.43364, 830.336, -878.858, 0.626324, -1611.27, 1659.17], 
 [-1.43364, 264.209, -282.28, 19.505, 2.19218e-14, -47.8954, 50.807, 1.0244e-13, 95.8864, -98.798], 
 [-282.28, 264.209, -1.43364, 2.19218e-14, 19.505, -3.97904e-13, -50.807, 47.8954, 98.798, -95.8864], 
 [-0.626324, -781.814, 830.336, -47.8954, -3.97904e-13, 152.048, -160.649, -4.68958e-13, -281.71, 290.311], 
 [878.858, 1.89806e-09, -878.858, 50.807, -50.807, -160.649, 339.694, -160.649, -9.19826, -9.19826], 
 [-830.336, 781.814, 0.626324, 1.0244e-13, 47.8954, -4.68958e-13, -160.649, 152.048, 290.311, -281.71], 
 [-1659.17, 3075.76, -1611.27, 95.8864, 98.798, -281.71, -9.

RuntimeError: Error in SX::gradient at .../casadi/core/generic_matrix.hpp:1468:
Error in SX::jtimes at .../casadi/core/generic_matrix.hpp:1456:
Error in Function::Function for 'reverse_temp' [SXFunction] at .../casadi/core/function.cpp:224:
.../casadi/core/function_internal.cpp:145: Error calling SXFunction::init for 'reverse_temp':
.../casadi/core/x_function.hpp:331: For reverse_temp: Xfunction input arguments must be purely symbolic.
Argument 0(i0) is not symbolic.